<a href="https://colab.research.google.com/github/sanntana21/TFG/blob/main/matrix_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREDICCIONES PARA TODA LA POBLACION SIN AGREGACION**

In [67]:
from statistics import mean, median
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  torch.set_printoptions(precision=decimals)
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [68]:
INDEXS = [{"train":[{"start":{"day":2,"hour":0,"minute":0},
                     "end":{"day":20,"hour":22,"minute":0}}
                    ],
           "validation":[{"start":{"day":22,"hour":0,"minute":0},
                     "end":{"day":24,"hour":22,"minute":0}}
                    ],
           "test":[{"start":{"day":26,"hour":0,"minute":0},
                     "end":{"day":27,"hour":22,"minute":0}}
                    ]}
          ]

In [69]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 1
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_RESULTS = True
LOW_DATA = True
SPLIT = 0
MULTI_STEP_FORECAST = False

In [70]:
if READ_LOCAL_DATA:
  PATH = "Resources/Individual/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Individual/"

if LOW_DATA:
    PATH += "LowData/"

In [71]:
import gzip
documents = ['minuteY','hourY','dayY']
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

NUMBER_OF_PARTICIPANTS = dataX.shape[0]
print('\033[1m' + "\t SHAPES" + '\033[0m')

	 SHAPES


In [72]:
if MULTI_STEP_FORECAST:
  if COMPUTED_OPTION == 0:
      dataX_nuevo = dataX
      JUMP = 1
  elif COMPUTED_OPTION == 1:
      JUMP = 60
      dataX_nuevo = np.ones(shape=(NUMBER_OF_PARTICIPANTS,dataX.shape[1],24))
      for k in range(0,NUMBER_OF_PARTICIPANTS):
          for i in range(0,dataX.shape[1]):
              for j in range(0,24):
                  dataX_nuevo[k,i,j] = np.sum(dataX[k,i,60*j:60*(j+1)])
  else:
      JUMP = 1440
      dataX_nuevo = np.ones(shape=(dataX.shape[0],1))
      for i in range(0,dataX.shape[0]):
          dataX_nuevo[i,:] = np.sum(dataX[i,:])


In [73]:
#We split a test set for testing
# train_test_split(dataX, dataY, test_size=TEST_SIZE)

def calculate_index(time):
  minute_index = time["day"] * 1440 + time["hour"]*60 + time["minute"]
  return minute_index

def get_split(dataX,dataY,index):
  start = calculate_index(index[0]["start"])
  end = calculate_index(index[0]["end"])
  X_split = dataX[:,start:end,:]
  y_split = dataY[:,start:end,:]
  if len(index) > 1:
    for i in range(1,index):
      start = calculate_index(index[i]["start"])
      end = calculate_index(index[i]["end"])
      X_split = np.concatenate(X_split,dataX[:,start:end,:])
      y_split = np.concatenate(y_split,dataY[:,start:end,:])
  return X_split,y_split

def train_test_validation_split(dataX,dataY,indexs):
  X_train,y_train = get_split(dataX,dataY,indexs["train"])
  X_validation,y_validation = get_split(dataX,dataY,indexs["validation"])
  X_test,y_test = get_split(dataX,dataY,indexs["test"])
  return X_train,y_train,X_validation,y_validation,X_test,y_test


X_train,y_train,X_validation,y_validation,X_test,y_test = train_test_validation_split(dataX,dataY,INDEXS[SPLIT])
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)

Examples for training
 X: (25, 27240, 1440) y: (25, 27240, 2)
Examples for validation
 X: (25, 4200, 1440) y: (25, 4200, 2)
Examples for test
 X: (25, 2760, 1440) y: (25, 2760, 2)


In [74]:
X_train,y_train,X_validation,y_validation,X_test,y_test = [ i.transpose(1,0,2) for i in
    [X_train,y_train,X_validation,y_validation,X_test,y_test ]
    ]
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for validation\n","X:",X_validation.shape,"y:",y_validation.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)

Examples for training
 X: (27240, 25, 1440) y: (27240, 25, 2)
Examples for validation
 X: (4200, 25, 1440) y: (4200, 25, 2)
Examples for test
 X: (2760, 25, 1440) y: (2760, 25, 2)


In [75]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[2]
WINDOW_SIZE = X_train.shape[2]

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(NUMBER_OF_PARTICIPANTS ,WINDOW_SIZE))
# x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128,return_sequences=True, activation="relu")(inputs) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

Model: "model_5_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 25, 1440)]        0         
                                                                 
 lstm_4 (LSTM)               (None, 25, 128)           803328    
                                                                 
 dense_4 (Dense)             (None, 25, 2)             258       
                                                                 
Total params: 803,586
Trainable params: 803,586
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [77]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train,
            y_train,
            epochs=3,
            verbose=1,
            batch_size=128,
            validation_data=(X_validation, y_validation))

Epoch 1/3
213/213 [==============================] - 27s 112ms/step - loss: 26.2145 - mae: 26.2145 - val_loss: 19.4915 - val_mae: 19.4915
Epoch 2/3
213/213 [==============================] - 24s 112ms/step - loss: 15.7234 - mae: 15.7234 - val_loss: 16.2261 - val_mae: 16.2261
Epoch 3/3
213/213 [==============================] - 24s 110ms/step - loss: 14.2100 - mae: 14.2100 - val_loss: 15.7788 - val_mae: 15.7788


In [78]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data,verbose=2)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)

87/87 - 3s - 3s/epoch - 36ms/step


In [79]:
#Se suman los mets de los participantes para realizar el estudio global
predictions = np.sum(predictions,axis=1)
y_test = np.sum(y_test,axis=1)
print(predictions.shape)
print(predictions.shape)

(2760, 2)
(2760, 2)


In [80]:
print("Resultados poblacionales valor a valor")
print('\033[1m' + "MSE: " + str(mean_squared_error(y_test,predictions)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test,predictions)) + '\033[0m')

Resultados poblacionales valor a valor
MSE: 63132.016
MAE: 179.15344


In [88]:
SPLIT_INTO_TWO_DAYS = True
if not MULTI_STEP_FORECAST:
  print("Resultados poblacionales cada dos horas")
  print('\033[1m' + "MSE: " + str(mean_squared_error(np.sum(y_test,axis=1),np.sum(predictions,axis=1))) + '\033[0m')
  print('\033[1m' + "MAE: " + str(mean_absolute_error(np.sum(y_test,axis=1),np.sum(predictions,axis=1))) + '\033[0m')

  list_of_MAE = [  mean_squared_error(predictions[i],y_test[i]) for i in range(0,len(y_test)) ]
  list_of_values = sorted(list_of_MAE)
  mean_value = mean(list_of_MAE)
  closest_value = min(list_of_MAE, key=lambda x: abs(x - mean_value))
  # Crear un array de índices
  indices = [list_of_MAE.index(list_of_values[-1]),
             list_of_MAE.index(closest_value) ,
            list_of_MAE.index(list_of_MAE[0])]
  if COMPUTED_OPTION == 0:
    for i in indices:
      plot_predictions_vs_real(predictions[i],y_test[i])
  else:
    for i in indices:
        END = 24
        STARTED_MINUTE = 0
        previous = np.ones(shape=(24))
        X_sum = np.sum(X_test,axis=1)
        for j in range(0,24):
            previous[j] = np.sum(X_sum[i,:][60*j:60*(j+1)])
        predictions_to_plot = np.ones(shape=(END + HORIZON))
        predictions_to_plot[0:END] = previous[:]
        predictions_to_plot[END:] = predictions[i,:]
        y_test_to_plot = np.ones(shape=(END + HORIZON))
        y_test_to_plot[0:END] = previous[:]
        y_test_to_plot[END:] = y_test[i,:]
        plot_predictions_vs_real(predictions_to_plot,y_test_to_plot)

  if SPLIT_INTO_TWO_DAYS:
      index = 0
      period = X_test[::120,:]
      period_results = make_preds(model_LSTM,period)
      period_results = np.sum(period_results,axis=1)
      period_results_to_plot = np.array(period_results).reshape(HORIZON*23)
      y_test_to_plot = y_test[::120,:].reshape(HORIZON*23)
      plot_predictions_vs_real(predictions=period_results_to_plot,reals=y_test_to_plot)
      print('\033[1m' + "Predicted: " + str(np.sum(period_results_to_plot)) + '\033[0m')
      print('\033[1m' + "Reals: " + str(np.sum(y_test_to_plot)) + '\033[0m')


Resultados poblacionales cada dos horas
MSE: 237794.44
MAE: 352.306


1/1 - 0s - 32ms/epoch - 32ms/step


Predicted: 98056.625
Reals: 105584.734


<Figure size 640x480 with 0 Axes>

In [82]:
if SAVE_RESULTS:
    file_path = 'Resources/Resultados/Matrix/'

    if LOW_DATA:
        file_path  += "LowData/"

    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))

In [83]:
if MULTI_STEP_FORECAST:
    day_before = np.array(X_test[0:1,:,:],dtype=np.float32)
    predicted_values_day_one = np.ones(shape=(WINDOW_SIZE,NUMBER_OF_PARTICIPANTS,1),dtype=np.float32)
    print(predicted_values_day_one.shape)
    print(day_before.shape)
    for i in range(0,WINDOW_SIZE):
        prediction = make_preds(model_LSTM, day_before)
        day_before[:,:,0:-1] = day_before[:,:,1:]
        day_before[:,:,-1] = prediction[:]
        predicted_values_day_one[i,:,0] = prediction[:]

In [84]:
if MULTI_STEP_FORECAST:
    SECOND_DAY = X_test.shape[2]
    day_before = np.array(X_test[SECOND_DAY:SECOND_DAY+1,:,:],dtype=np.float32)
    predicted_values_day_two = np.ones(shape=(WINDOW_SIZE,NUMBER_OF_PARTICIPANTS,1),dtype=np.float32)
    for i in range(0,WINDOW_SIZE):
        prediction = make_preds(model_LSTM, day_before)
        day_before[:,:,0:-1] = day_before[:,:,1:]
        day_before[:,:,-1] = prediction[:]
        predicted_values_day_two[i,:,0] = prediction[:]

In [85]:
if MULTI_STEP_FORECAST:
    poblational_predicted_values_day_one = np.sum(predicted_values_day_one,axis=1)
    poblational_predicted_values_day_two = np.sum(predicted_values_day_two,axis=1)
    poblational_y_test = np.sum(y_test,axis=1)

In [86]:
if MULTI_STEP_FORECAST:
    print("Predecidos : " + str(int(np.sum(poblational_predicted_values_day_one[:,0]))))
    print("Reales : " + str(int(np.sum(poblational_y_test[0:WINDOW_SIZE,0]))))
    if COMPUTED_OPTION < 2:
        plot_predictions_vs_real(poblational_predicted_values_day_one[:,0],poblational_y_test[0:WINDOW_SIZE,0])

In [87]:
if MULTI_STEP_FORECAST:
    print("Predecidos : " + str(int(np.sum(poblational_predicted_values_day_two[:,0]))))
    print("Reales : " + str(int(np.sum(poblational_y_test[SECOND_DAY:,0]))))
    if COMPUTED_OPTION < 2:
        plot_predictions_vs_real(poblational_predicted_values_day_two[:100,0],poblational_y_test[SECOND_DAY:SECOND_DAY+100,0])